In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from scipy.sparse import csr_matrix
import os
#print(os.listdir("../input"))
path = "C:/Users/rarora17/git/mlcourse.ai_Assignments/data/Catchmeifyoucan"
print(os.listdir(path))
# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'site_dic.pkl', 'test_sessions.csv', 'train', 'train.zip', 'train_sessions.csv']


In [2]:
data_train = pd.read_csv(path+'/train_sessions.csv')
data_test = pd.read_csv(path+'/test_sessions.csv')

In [3]:
data_train.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
0,1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
2,3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
3,4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
4,5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [4]:
data_train.columns

Index(['session_id', 'site1', 'time1', 'site2', 'time2', 'site3', 'time3',
       'site4', 'time4', 'site5', 'time5', 'site6', 'time6', 'site7', 'time7',
       'site8', 'time8', 'site9', 'time9', 'site10', 'time10', 'target'],
      dtype='object')

##### Convert the 'time1','time2','time3'..........'time10' to 'date time' format

In [5]:
times = ['time%s' %i for i in np.arange(1,11)]

In [6]:
data_test[times] = data_test[times].apply(pd.to_datetime)
data_train[times] = data_train[times].apply(pd.to_datetime)

In [7]:
data_train =  data_train.sort_values(by='time1')

In [8]:
data_train.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
21668,21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54842,54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77291,77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114020,114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146669,146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


##### Change site1, ..., site10 columns type to integer and fill NA-values with zeros

In [9]:
sites = ['site%s' %i for i in np.arange(1,11)]


In [10]:
data_train[sites] = data_train[sites].fillna(0).astype('int')
#.to_csv('train_sites',header=None,index=None,sep=' ')
data_test[sites] = data_test[sites].fillna(0).astype('int')
#.to_csv('test_sites',header=None,index=None,sep=' ')

##### Read the dictionary and create a dataframe

In [11]:
with open(path+"/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)
# Create dataframe for the dictionary
sites_dict = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), columns=['site'])


In [12]:
site_dict = pd.DataFrame(list(site_dict.keys()),index=list(site_dict.values()),columns=['sites'])
print('Websites total:', sites_dict.shape[0])
sites_dict.head()

Websites total: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


##### Concatenate the train and test data 

In [13]:
# Output 
y_train = data_train['target']

# United data frame to generate teh feature set
full_df = pd.concat([data_train.drop(['target'],axis=1),data_test])

# Index Location of end of train and start of test data 
idx_split = data_train.shape[0]
print(idx_split)

253561


In [14]:
#Data frame with indeces of visited websites in session
full_sites =  full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
21668,56,55,0,0,0,0,0,0,0,0
54842,56,55,56,55,0,0,0,0,0,0
77291,946,946,951,946,946,945,948,784,949,946
114020,945,948,949,948,945,946,947,945,946,946
146669,947,950,948,947,950,952,946,951,946,947


In [15]:
# Sequence of Indecies
sites_flatten = full_sites.values.flatten()

full_sites_sparse =  csr_matrix(([1]*sites_flatten.shape[0],
                                  sites_flatten,
                                  range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

In [16]:
sites_flatten[0:10],sites_flatten[10:20],sites_flatten[20:30]

(array([56, 55,  0,  0,  0,  0,  0,  0,  0,  0]),
 array([56, 55, 56, 55,  0,  0,  0,  0,  0,  0]),
 array([946, 946, 951, 946, 946, 945, 948, 784, 949, 946]))

In [18]:
print(full_sites_sparse[0:2,:])

  (0, 55)	1
  (0, 54)	1
  (1, 55)	1
  (1, 54)	1
  (1, 55)	1
  (1, 54)	1


## Train Logistic Regression Model

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,StratifiedKFold

In [20]:
logit = LogisticRegression()

In [21]:
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
21668,56,55,0,0,0,0,0,0,0,0
54842,56,55,56,55,0,0,0,0,0,0
77291,946,946,951,946,946,945,948,784,949,946
114020,945,948,949,948,945,946,947,945,946,946
146669,947,950,948,947,950,952,946,951,946,947


In [22]:
X_train = full_sites_sparse[:idx_split,:]
X_test = full_sites_sparse[idx_split:,:]

In [23]:
%%time
cv_scores = cross_val_score(logit,X_train,y_train,cv=5,scoring='roc_auc')

Wall time: 20.5 s


In [24]:
print("cv_scores",cv_scores,cv_scores.mean())

cv_scores [ 0.91327204  0.83213265  0.87783231  0.89247358  0.91521577] 0.886185268661


In [25]:
logit.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
test_pred_logit = logit.predict_proba(X_test)[:,1]

In [27]:
def  write_result_to_submission( predicted_labels,out_file,target="target",index_label="session_id"):
    predicted_df = pd.DataFrame( predicted_labels, 
                                index =  np.arange(1,predicted_labels.shape[0]+1),
                                columns=[target])
    predicted_df.to_csv(out_file,index_label=index_label)    


In [28]:
test_pred_logit.shape

(82797,)

In [29]:
write_result_to_submission(test_pred_logit,'baseline.csv')